In [ ]:
from datetime import datetime, timedelta
import os
import sys

import numpy as np
import pandas as pd
import laspy

In [ ]:
root = "/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD/Toulouse"

list_laz = [laz_file for laz_file in os.listdir(root) if laz_file[-4:]==".laz"]
len(list_laz)

# Acquisition time
The documentation (https://geoservices.ign.fr/sites/default/files/2023-10/DC_LiDAR_HD_1-0_PTS.pdf Section 2.2.12) states that:

"La valeur du temps (gps_time) du point correspond au nombre de seconde écoulées depuis le
14/09/2011 à 00:00:00 UTC"

In [ ]:
source_time = datetime(2011, 9, 14, 0, 0, 0)
n_laz = len(list_laz)

df = pd.DataFrame(columns=["laz_file", "point_count", "min_time", "max_time", "med_time"])

for i, laz_file in enumerate(list_laz):
    print(f"\rReading {i+1}/{n_laz} laz file", end="")
    laz = laspy.read(os.path.join(root, laz_file))
    # list(laz_1.point_format.dimension_names)
    pc = laz.header.point_count
    
    min_delta = np.min(laz['gps_time'])
    max_delta = np.max(laz['gps_time'])
    med_delta = np.median(laz['gps_time'])
    
    min_time = source_time + timedelta(seconds=np.floor(min_delta))
    max_time = source_time + timedelta(seconds=np.ceil(max_delta))
    med_time = source_time + timedelta(seconds=np.ceil(med_delta))
    
    df.loc[i, ["laz_file", "point_count", "min_time", "max_time", "med_time"]] = [laz_file, pc, min_time, max_time, med_time]

# Aggregate CSV

In [ ]:
cols = ["laz_file", "point_count", "min_time", "max_time", "med_time", "med_x", "med_y", "min_x", "min_y", "max_x", "max_y"]
df = pd.DataFrame(columns=cols)

for i, laz_file in enumerate(list_laz):
    df_tmp = pd.read_csv(os.path.join(root, "csv_data", laz_file + ".csv"), index_col=0)
    df.loc[i, cols] = df_tmp.loc[0, cols]
df.set_index("laz_file", inplace=True)

In [ ]:
df["med_time"].astype('datetime64[ns]').quantile(0.5, interpolation="midpoint")

# Fuse LAZ

In [ ]:
output_path = os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/LiDAR_HD/", "Ernest_Wallon.laz")
list_files = ["LHD_FXX_0572_6282_PTS_C_LAMB93_IGN69.copc.laz", "LHD_FXX_0571_6282_PTS_C_LAMB93_IGN69.copc.laz"]
[k in list_laz for k in list_files]

In [ ]:
from laspy import CopcReader

scales = None
offsets = None
mode = "w"

crdr = CopcReader.open(os.path.join(root, list_files[0])).header
new_header = laspy.LasHeader(version=crdr.version, point_format=crdr.point_format) #Creating a header wich is not COPC
print(new_header.point_count)
total_point_count = 0
for file in list_files:
    print(file)
    with CopcReader.open(os.path.join(root, file)) as laz_to_add:
        if scales is None:
            scales = laz_to_add.header.scales
            offsets = laz_to_add.header.offsets
        elif (scales != laz_to_add.header.scales).any() or (offsets != laz_to_add.header.offsets).any():
            raise RuntimeError(f"Scales or offsets are not consistent between laz files\nScales: {scales}  vs {laz_to_add.header.scales}\nOffsets: {offsets} | {laz_to_add.header.offsets}")
        points = laz_to_add.query()  # Query all points

    with laspy.open(output_path, mode=mode, header=new_header) as out:
        if mode=="w":
            out.write_points(points)
            mode = "a"
        else:
            out.append_points(points)

In [ ]:
laz_to_add = laspy.read(output_path)
laz_to_add.header.point_count

In [ ]:
print(df.loc[list_files]["point_count"].sum())
df.loc[list_files]

In [ ]:
points

In [ ]:
laz_to_add.header.max_point_count()

In [ ]:
points

In [ ]:
print(new_header.vlrs)

In [ ]:
new_header